## Persistent Volume

In [1]:
%%writefile pv_dags.yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-dags
  labels:
    type: local
spec:
  storageClassName: microk8s-hostpath
  accessModes: ["ReadWriteMany"]
  capacity:
    storage: 50Gi
  hostPath:
    type: DirectoryOrCreate
    path: "/home/meng/Work/Airflow/dags"

Writing pv_dags.yaml


In [2]:
%%writefile pvc_dags.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-dags
  namespace: airflow
spec:
  storageClassName: microk8s-hostpath
  volumeName: airflow-dags
  accessModes: ["ReadWriteMany"]
  resources:
    requests:
      storage: 50Gi

Writing pvc_dags.yaml


In [3]:
%%writefile create.sh
#!/bin/bash

microk8s helm3 repo add apache-airflow https://airflow.apache.org
microk8s helm3 repo update

microk8s kubectl create namespace airflow
microk8s kubectl create -f pv_dags.yaml
microk8s kubectl create -f pvc_dags.yaml

Writing create.sh


## Install

In [4]:
%%writefile override.yaml

# Images
images:
  airflow:
    repository: darklemon/airflow
    tag: latest
    pullPolicy: Always

# Webserver secret key
webserverSecretKey: $(python3 -c 'import secrets; print(secrets.token_hex(16))')

# Executor
executor: CeleryKubernetesExecutor
logs:
  persistence:
    enabled: true

# dags
dags:
  persistence:
    enabled: true
    existingClaim: airflow-dags

Writing override.yaml


In [5]:
%%writefile Dockerfile
FROM apache/airflow:latest

RUN python3 -m pip install --no-cache-dir -U pip wheel \
    && pip install --no-cache-dir -U \
        scrapy \
    && pip check

Writing Dockerfile


In [6]:
%%writefile build.sh
#!/bin/bash

docker build . --no-cache --pull \
-f Dockerfile \
-t darklemon/airflow:latest

docker push darklemon/airflow:latest

microk8s helm3 upgrade \
--install airflow apache-airflow/airflow \
--namespace airflow \
-f override.yaml

docker rmi darklemon/airflow:latest
docker rmi apache/airflow:latest

Writing build.sh


## Webserver

In [1]:
%%writefile ~/work/Airflow/run_webserver.sh
#!/bin/bash

microk8s kubectl port-forward svc/airflow-webserver 8080:8080 --namespace airflow

Writing /home/meng/work/Airflow/run_webserver.sh


## Dags - example

In [ ]:
%%writefile ~/work/Airflow/dags/tutorial.py
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

yesterday = datetime.datetime.combine(
    datetime.datetime.today() - datetime.timedelta(1),
    datetime.datetime.min.time())

with DAG(
    dag_id="tutorial",
    start_date=yesterday,
    schedule_interval="@hourly"
) as dag:

    t1 = BashOperator(
        task_id="echo",
        bash_command="echo $PWD"
    )
    t1

## Uninstall

In [1]:
%%writefile uninstall.sh
#!/bin/bash

microk8s helm3 delete airflow \
--namespace airflow
microk8s helm3 repo rm apache-airflow
microk8s kubectl delete -f pv_dags.yaml
microk8s kubectl delete -f pvc_dags.yaml
microk8s kubectl delete ns airflow
microk8s ctr images rm $(microk8s ctr images ls name~='airflow' | awk {'print $1'})

Overwriting uninstall.sh
